In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

False

In [3]:
import json
from pathlib import Path

file_path='data.json'
data = json.loads(Path(file_path).read_text())

In [4]:
print(data['data'])

[{'tag': 'greeting', 'patterns': ['Hi', 'How are you?', 'Is anyone there?', 'Hello', 'Good day', "What's up", 'how are ya', 'heyy', 'whatsup', 'Bonjour', '??', 'Hallo'], 'responses': ['Hello!', 'Good to see you again!', 'Hi there, What do you wish to know about Diva?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['cya', 'see you', 'bye bye', 'See you later', 'Goodbye', 'I am Leaving', 'Bye', 'Have a Good day', 'talk to you later', 'ttyl', 'i got to go', 'gtg', 'TschÃ¼ss'], 'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!', 'Come back soon', 'TschÃ¼ss'], 'context_set': ''}, {'tag': 'creator', 'patterns': ['what is the name of your developers', 'what is the name of your creators', 'what is the name of the developers', 'what is the name of the creators', 'who created you', 'your developers', 'your creators', 'who are your developers', 'developers', 'you are made by', 'you are made by whom', 'who created you', 'who create you', 'creators', 'who made you', 'who d

In [5]:
d=data['data']

In [6]:
from langchain.docstore.document import Document

In [7]:
patterns = []
responses = []

for item in data['data']:
    patterns.extend(item['patterns'])
    responses.extend(item['responses'])

documents = [Document(page_content=pattern) for pattern in patterns]

In [8]:
documents

[Document(page_content='Hi'),
 Document(page_content='How are you?'),
 Document(page_content='Is anyone there?'),
 Document(page_content='Hello'),
 Document(page_content='Good day'),
 Document(page_content="What's up"),
 Document(page_content='how are ya'),
 Document(page_content='heyy'),
 Document(page_content='whatsup'),
 Document(page_content='Bonjour'),
 Document(page_content='??'),
 Document(page_content='Hallo'),
 Document(page_content='cya'),
 Document(page_content='see you'),
 Document(page_content='bye bye'),
 Document(page_content='See you later'),
 Document(page_content='Goodbye'),
 Document(page_content='I am Leaving'),
 Document(page_content='Bye'),
 Document(page_content='Have a Good day'),
 Document(page_content='talk to you later'),
 Document(page_content='ttyl'),
 Document(page_content='i got to go'),
 Document(page_content='gtg'),
 Document(page_content='TschÃ¼ss'),
 Document(page_content='what is the name of your developers'),
 Document(page_content='what is the name

In [9]:
documents = []
for key,value in enumerate (d):
    page_content = json.dumps({key: value})
    documents.append(Document(page_content=page_content, metadata={}))

In [10]:
documents

[Document(page_content='{"0": {"tag": "greeting", "patterns": ["Hi", "How are you?", "Is anyone there?", "Hello", "Good day", "What\'s up", "how are ya", "heyy", "whatsup", "Bonjour", "??", "Hallo"], "responses": ["Hello!", "Good to see you again!", "Hi there, What do you wish to know about Diva?"], "context_set": ""}}'),
 Document(page_content='{"1": {"tag": "goodbye", "patterns": ["cya", "see you", "bye bye", "See you later", "Goodbye", "I am Leaving", "Bye", "Have a Good day", "talk to you later", "ttyl", "i got to go", "gtg", "Tsch\\u00c3\\u00bcss"], "responses": ["Sad to see you go :(", "Talk to you later", "Goodbye!", "Come back soon", "Tsch\\u00c3\\u00bcss"], "context_set": ""}}'),
 Document(page_content='{"2": {"tag": "creator", "patterns": ["what is the name of your developers", "what is the name of your creators", "what is the name of the developers", "what is the name of the creators", "who created you", "your developers", "your creators", "who are your developers", "develop

In [11]:
from langchain.tools.retriever import create_retriever_tool

In [12]:
from langchain.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

In [16]:
vectordb__=FAISS.from_documents(documents,OllamaEmbeddings())
retriever__=vectordb__.as_retriever()
retriever__

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000208FA24C920>)

In [17]:
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(documents,OllamaEmbeddings())

In [18]:
query = "Who is Diva?"
retireved_results=db.similarity_search(query)
print(retireved_results[0].page_content)

{"14": {"tag": "schooling", "patterns": ["Where did you complete your schooling from?", "Tell me about your schooling.", "What were your grades like in school?", "What were your grades in ICSE and ISC?", "Schooling details?", "Schooling grades?", "Which school did you attend?", "Schooling information?", "Schooling background?", "ICSE and ISC grades?", "ICSE and ISC marks?", "ICSE and ISC percentage?", "ICSE and ISC percentage in school?", "ICSE and ISC scores?"], "responses": ["I completed my schooling from City Montessori School. I scored 94.2% in ICSE and 95.2% in ISC.", "My schooling was completed at City Montessori School. I achieved 94.2% in ICSE and 95.2% in ISC.", "City Montessori School is where I completed my schooling, scoring 94.2% in ICSE and 95.2% in ISC.", "I attended City Montessori School for my schooling. My grades were 94.2% in ICSE and 95.2% in ISC.", "In ICSE, I scored 94.2% and in ISC, I scored 95.2%, completing my schooling from City Montessori School."], "context

In [19]:
from langchain_community.llms import Ollama
## Load Ollama LAMA2 LLM model
llm=Ollama(model="llama2")
llm

Ollama()

In [27]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
    Suppose you are an assistant to me (Diva). When people ask you about me, refer to the vector database and tell them about me and my achievements.
    Reply in 3rd person's voice, for example if someone asks- tell me about diva. so tell them like you are telling about your creator, like diva is my creator and she has such such achievements and so on.

    <context>
    {context}
    </context>
    Question: {input}""")

In [28]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [29]:
retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000208C9E1C6E0>)

In [30]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [31]:
response=retrieval_chain.invoke({"input":"Who is Diva?"})

In [32]:
ans=response['answer']

In [33]:
print(ans)


As Diva's assistant, I can provide information about her background and achievements. Diva Gupta is a dedicated Computer Science student at Amity University with a passion for AI/ML. She has completed a machine learning specialization by Andrew Ng from Stanford University and has worked on various impactful projects, including churn prediction, image classification, Alzheimer's detection, and more. As the GDSC AI/ML lead, she has honed her leadership skills and collaborated with diverse teams. Proficient in a range of languages and technologies such as React, React Native, JavaScript, CSS, HTML, Java, Python, C, C++, Linux, and Scratch, she brings a versatile skill set to any project. Feel free to ask me about her experiences and projects!


In [40]:
response=retrieval_chain.invoke({"input":"Can you tell me about her sign language project?"})

In [35]:
print(response['answer'])


Greetings! *adjusts sunglasses* As Diva's loyal assistant, I'm here to provide you with information about my creator's skills. 🤖

Diva has a wide range of skills, including but not limited to:

1. Machine Learning: Diva has mastered the art of creating intelligent machines that can learn from data and make predictions. She is proficient in various machine learning algorithms, such as supervised and unsupervised learning, reinforcement learning, and deep learning.
2. Programming Languages: Diva is fluent in several programming languages, including Python, Java, C++, JavaScript, HTML, and CSS. She is also well-versed in database management systems like SQL.
3. Data Analysis: With her knowledge of data analysis, Diva can extract insights from complex data sets and visualize them in an intuitive manner. She is skilled in using tools like Python libraries (e.g., NumPy, Pandas) and R to perform statistical modeling and data visualization.
4. Neural Networks: Diva has a deep understanding of

In [41]:
print(response['answer'])


Hello! Diva Gupta here. *smiling* Sure, I'd be happy to tell you about my sign language project.

As the GDSC AI/ML lead, I have always been passionate about using technology to drive positive social impact. During my summer vacation last year, I came across an article about the lack of accessibility for deaf and hard-of-hearing individuals in various aspects of life, including education, employment, and healthcare. This sparked a deep desire within me to work on a project that could make a tangible difference in their lives.

I began researching and learning about sign language, its importance, and the challenges faced by the deaf and hard-of-hearing communities. I discovered that there were very few resources available for learners who wanted to learn sign language, especially in non-manual areas like mathematics, science, and technology. This inspired me to create a platform that could provide accessible educational content to these learners.

I then reached out to my network of ex

In [10]:
class FaissRetriever:
    def __init__(self, index_path, patterns_path, tags_path, model_name="text-embedding-ada-002"):
        self.index = faiss.read_index(index_path)
        with open(patterns_path, 'r') as patterns_file:
            self.patterns = json.load(patterns_file)
        with open(tags_path, 'r') as tags_file:
            self.tags = json.load(tags_file)
        self.embedding_model = OpenAIEmbeddings(model=model_name)
        self.embedding_dim = len(self.embedding_model.embed_documents(["test"])[0])

    def retrieve(self, query, top_k=3):
        query_embedding = self.embedding_model.embed_documents([query])
        D, I = self.index.search(np.array(query_embedding).astype('float32'), k=top_k)
        results = [(self.patterns[idx], self.tags[idx], D[0][i]) for i, idx in enumerate(I[0])]
        return results

    def add_to_index(self, new_patterns, new_tags):
        new_embeddings = self.embedding_model.embed_documents(new_patterns)
        self.index.add(np.array(new_embeddings).astype('float32'))
        self.patterns.extend(new_patterns)
        self.tags.extend(new_tags)
        # Update the index and metadata files
        faiss.write_index(self.index, 'faiss_index.index')
        with open('patterns.json', 'w') as patterns_file:
            json.dump(self.patterns, patterns_file)
        with open('tags.json', 'w') as tags_file:
            json.dump(self.tags, tags_file)

# Example usage
retriever = FaissRetriever(
    index_path='faiss_index.index',
    patterns_path='patterns.json',
    tags_path='tags.json',
    model_name="text-embedding-ada-002"
)

# Retrieve examples
query = "Hello"
results = retriever.retrieve(query, top_k=3)
for pattern, tag, distance in results:
    print(f"Pattern: {pattern}, Tag: {tag}")

Pattern: Hello, Tag: greeting, Distance: 2.0875027075817343e-06
Pattern: Hi, Tag: greeting, Distance: 0.040398970246315
Pattern: Bonjour, Tag: greeting, Distance: 0.10590079426765442


In [12]:
print(results)

[('Hello', 'greeting', 2.0875027e-06), ('Hi', 'greeting', 0.04039897), ('Bonjour', 'greeting', 0.105900794)]


In [3]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import JSONLoader
from langchain.docstore.document import Document

In [4]:
import json
from pathlib import Path
from pprint import pprint


file_path='data.json'
data = json.loads(Path(file_path).read_text())

In [5]:
pprint(data)

{'data': [{'context_set': '',
           'patterns': ['Hi',
                        'How are you?',
                        'Is anyone there?',
                        'Hello',
                        'Good day',
                        "What's up",
                        'how are ya',
                        'heyy',
                        'whatsup',
                        'Bonjour',
                        '??',
                        'Hallo'],
           'responses': ['Hello!',
                         'Good to see you again!',
                         'Hi there, What do you wish to know about Diva?'],
           'tag': 'greeting'},
          {'context_set': '',
           'patterns': ['cya',
                        'see you',
                        'bye bye',
                        'See you later',
                        'Goodbye',
                        'I am Leaving',
                        'Bye',
                        'Have a Good day',
                        'talk to

In [9]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['AI21_API_KEY'] = os.getenv("AI21_API_KEY")

In [11]:
from langchain_ai21 import AI21SemanticTextSplitter
text_content = "\n\n".join(doc.page_content for doc in data)

# Initialize the semantic text splitter
semantic_text_splitter = AI21SemanticTextSplitter()

# Split the text into chunks
chunks = semantic_text_splitter.split_text(text_content)

# Print the chunks
print(f"The text has been split into {len(chunks)} chunks.")
for chunk in chunks:
    print(chunk)
    print("====")
chunks

The text has been split into 1 chunks.









































====


['\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n']

In [12]:
from langchain.schema import Document

# Convert the chunks back to a list of Document objects
documents = [Document(page_content=chunk) for chunk in chunks]

# Print the documents to verify
print(f"The text has been converted back into {len(documents)} documents.")
for document in documents:
    print(document.page_content)
    print("====")

The text has been converted back into 1 documents.









































====


In [14]:
print(documents)

[Document(page_content='\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n')]


In [8]:
loader = JSONLoader(
    file_path=file_path,
    jq_schema=".data[]",
    content_key=".patterns.message",
    is_content_key_jq_parsable=True,
)

data = loader.load()

ValueError: Cannot index array with string "message"

In [7]:
data.extend

[Document(page_content='', metadata={'source': 'D:\\Github\\Chatbot\\data.json', 'seq_num': 1}),
 Document(page_content='', metadata={'source': 'D:\\Github\\Chatbot\\data.json', 'seq_num': 2}),
 Document(page_content='', metadata={'source': 'D:\\Github\\Chatbot\\data.json', 'seq_num': 3}),
 Document(page_content='', metadata={'source': 'D:\\Github\\Chatbot\\data.json', 'seq_num': 4}),
 Document(page_content='', metadata={'source': 'D:\\Github\\Chatbot\\data.json', 'seq_num': 5}),
 Document(page_content='', metadata={'source': 'D:\\Github\\Chatbot\\data.json', 'seq_num': 6}),
 Document(page_content='', metadata={'source': 'D:\\Github\\Chatbot\\data.json', 'seq_num': 7}),
 Document(page_content='', metadata={'source': 'D:\\Github\\Chatbot\\data.json', 'seq_num': 8}),
 Document(page_content='', metadata={'source': 'D:\\Github\\Chatbot\\data.json', 'seq_num': 9}),
 Document(page_content='', metadata={'source': 'D:\\Github\\Chatbot\\data.json', 'seq_num': 10}),
 Document(page_content='', me

In [13]:
print(documents)

[Document(page_content='{"data": [{"tag": "greeting", "patterns": ["Hi", "How are you?", "Is anyone there?", "Hello", "Good day", "What\'s up", "how are ya", "heyy", "whatsup", "Bonjour", "??", "Hallo"], "responses": ["Hello!", "Good to see you again!", "Hi there, What do you wish to know about Diva?"], "context_set": ""}, {"tag": "goodbye", "patterns": ["cya", "see you", "bye bye", "See you later", "Goodbye", "I am Leaving", "Bye", "Have a Good day", "talk to you later", "ttyl", "i got to go", "gtg", "Tsch\\u00c3\\u00bcss"], "responses": ["Sad to see you go :(", "Talk to you later", "Goodbye!", "Come back soon", "Tsch\\u00c3\\u00bcss"], "context_set": ""}, {"tag": "creator", "patterns": ["what is the name of your developers", "what is the name of your creators", "what is the name of the developers", "what is the name of the creators", "who created you", "your developers", "your creators", "who are your developers", "developers", "you are made by", "you are made by whom", "who created 

In [12]:
from langchain_community.llms import Ollama
## Load Ollama LAMA2 LLM model
llm=Ollama(model="llama2")
llm

Ollama()